# IBM Applied Data Science Capstone Course by Coursera

### OPENING A NEW SHOPPING MALL IN KOLKATA, INDIA

#### 1. Import Libraries

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### 2. Get Neighbourhood Data from Wikipedia

In [53]:
#send the GET request - PAGE 1
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Kolkata").text

In [54]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html5lib')

In [55]:
import csv
#Initializing the csv_writer object and writing the name of the columns on it as the first row
csv_file = open('kolkata.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Neighbourhood'])

15

In [56]:
mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg) # Gets the length of number of `mw-category-groups` present

for i in range(1, length):  # Gets all the neighbourhoods
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') # Gets the title of the neighbourhood
        csv_writer.writerow([nbd]) # Writes the name of the neighbourhood in the csv file

In [57]:
#send the GET request - PAGE 2
data = requests.get("https://en.wikipedia.org/w/index.php?title=Category:Neighbourhoods_in_Kolkata&pagefrom=Maniktala#mw-pages").text
soup = BeautifulSoup(data, 'html5lib')

mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg)-1 # Gets the length of number of `mw-category-groups` present

for i in range(0, length):  # Gets all the neighbourhoods
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') # Gets the title of the neighbourhood
        csv_writer.writerow([nbd]) # Writes the name of the neighbourhood in the csv file

csv_file.close()

####  3. Creating the dataframe

In [58]:
kl_df = pd.read_csv('kolkata.csv')

In [60]:
kl_df.shape

(323, 1)

In [61]:
kl_df.head(10)

,Neighbourhood
0,Abhirampur
1,Agarpara
2,Ajoy Nagar
3,Alipore
4,Amodghata
5,Amtala
6,"Anandapur, Kolkata"
7,Ankurhati
8,Argari
9,Asuti


#### 4. Getting the Geographical coordinates

In [67]:
import geocoder # to get coordinates

In [68]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kolkata,West Bengal,India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [69]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighbourhood"].tolist() ]

In [72]:
len(coords)

323

#### 5. Adding Lat and Long to the DataFrame

In [74]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [76]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [78]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(323, 3)


In [79]:
# save the DataFrame as CSV file
kl_df.to_csv("kolkata_final.csv", index=False)

#### 6. Create a map of Kolkata with Neighbourhoods

In [80]:
# get the coordinates of Kolkata, West Bengal 
address = 'Kolkata, West Bengal, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kolkata, West Bengal, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kolkata, West Bengal, India 22.54541245, 88.3567751581234.


In [82]:
# create map of Kolkata using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [83]:
# save the map as HTML file
map_kl.save('map_kl.html')

#### 7. Using Foursquare API to explore the neighbourhoods

In [88]:
#FOURSQUARE API DETAILS
CLIENT_ID = 'OEREGCPD322NOBOGI0I4CMLXAGSLBQHTO4HIYRHNMR3TPP4Z' # your Foursquare ID
CLIENT_SECRET = 'OPGZYR20LJ4E0HOTQXBN234IQ144QCMZCL501USAY3EQI5A3' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OEREGCPD322NOBOGI0I4CMLXAGSLBQHTO4HIYRHNMR3TPP4Z
CLIENT_SECRET:OPGZYR20LJ4E0HOTQXBN234IQ144QCMZCL501USAY3EQI5A3


<b> Now, Let's get the top 30 venues that are within a radius of 1000 meters </b>

In [89]:
radius = 1000
venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [90]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2313, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abhirampur,22.530694,88.346503,Balaram Mullick & Radharaman Mullick,22.533097,88.347082,Indian Sweet Shop
1,Abhirampur,22.530694,88.346503,Jai Hind Dhaba,22.533109,88.353268,Dhaba
2,Abhirampur,22.530694,88.346503,Balwant Singh's Eating House,22.537714,88.344220,Dhaba
3,Abhirampur,22.530694,88.346503,Oh! Calcutta,22.538357,88.351406,Bengali Restaurant
4,Abhirampur,22.530694,88.346503,Red Hot Chilli Pepper,22.529016,88.355805,Chinese Restaurant


<b> Let's check how many venues were returned for each neighorhood </b>

In [92]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abhirampur,20,20,20,20,20,20
Agarpara,6,6,6,6,6,6
Ajoy Nagar,7,7,7,7,7,7
Alipore,9,9,9,9,9,9
Amtala,5,5,5,5,5,5
"Anandapur, Kolkata",4,4,4,4,4,4
Ankurhati,5,5,5,5,5,5
Argari,11,11,11,11,11,11
B. B. D. Bagh,18,18,18,18,18,18


<b>Let's find out how many unique categories can be curated from all the returned venues</b>

In [93]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 155 uniques categories.


In [94]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Sweet Shop', 'Dhaba', 'Bengali Restaurant',
       'Chinese Restaurant', 'Café', 'Nightclub', 'Hotel',
       'Fast Food Restaurant', 'American Restaurant', 'Shopping Mall',
       'Multiplex', 'Italian Restaurant', 'Tea Room', 'Ice Cream Shop',
       'Department Store', 'South Indian Restaurant', 'ATM',
       'Train Station', 'Grocery Store', 'Bus Station', 'Bakery',
       'Dessert Shop', 'Pizza Place', 'Pharmacy', 'Mobile Phone Shop',
       'Asian Restaurant', 'Park', 'Plaza', 'Juice Bar', 'Platform',
       'Pool', 'Cricket Ground', 'Indian Restaurant', 'Stadium',
       'Neighborhood', 'Thai Restaurant', 'Mughlai Restaurant', 'River',
       'Boat or Ferry', 'Breakfast Spot', 'Bank', 'Market',
       'Metro Station', 'Vegetarian / Vegan Restaurant', 'Bistro',
       'Fried Chicken Joint', 'Supermarket', 'Watch Shop',
       'Clothing Store', 'Taxi Stand'], dtype=object)

In [103]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

#### 8. Analyze Each Neighbourhood

In [105]:

# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + kl_onehot.columns[:-1].values.tolist()
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2313, 156)


,Neighborhoods,ATM,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Awadhi Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Cricket Ground,Department Store,Dessert Shop,Dhaba,Diner,Dumpling Restaurant,Electronics Store,Event Service,Falafel Restaurant,Fast Food Restaurant,Field,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Pool,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Military Base,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Neighborhood,Nightclub,North Indian Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Port,Pub,Racetrack,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Tibetan Restaurant,Toll Booth,Tourist Information Center,Train Station,Tram Station,Used Bookstore,Vegetarian / Vegan Restaurant,Watch Shop,Women's Store,Zoo
0,Abhirampur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abhirampur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abhirampur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abhirampur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Abhirampur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<b>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</b>

In [106]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped.head()

(283, 156)


,Neighborhoods,ATM,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Awadhi Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Cricket Ground,Department Store,Dessert Shop,Dhaba,Diner,Dumpling Restaurant,Electronics Store,Event Service,Falafel Restaurant,Fast Food Restaurant,Field,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Pool,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Military Base,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Neighborhood,Nightclub,North Indian Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Port,Pub,Racetrack,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Tibetan Restaurant,Toll Booth,Tourist Information Center,Train Station,Tram Station,Used Bookstore,Vegetarian / Vegan Restaurant,Watch Shop,Women's Store,Zoo
0,Abhirampur,0.000000,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.100000,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0,0.050000,0.000000,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.05,0.0,0.05,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Agarpara,0.833333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0
2,Ajoy Nagar,0.142857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142

In [108]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

36

<b>Create a new DataFrame for Shopping Mall data only</b>

In [109]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [114]:
kl_mall.to_csv('kolkata_mall_data.csv')

#### 9. Cluster Neighbourhoods

In [115]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [116]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [117]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Abhirampur,0.050000,2
1,Agarpara,0.000000,0
2,Ajoy Nagar,0.142857,1
3,Alipore,0.000000,0
4,Amtala,0.000000,0


In [123]:
# merge toronto_grouped with kolkata_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighbourhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(283, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abhirampur,0.050000,2,22.530694,88.346503
1,Agarpara,0.000000,0,22.684050,88.391650
2,Ajoy Nagar,0.142857,1,22.489660,88.396400
3,Alipore,0.000000,0,22.526600,88.335100
4,Amtala,0.000000,0,22.482678,88.376588


In [124]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(283, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
141,Kamalgazi,0.000000,0,22.447295,88.390689
174,Mankundu,0.000000,0,22.471185,88.395153
175,"Model Town, Kolkata",0.000000,0,22.463000,88.395820
176,"Mukundapur, Kolkata",0.000000,0,22.589113,88.355020
177,Nabagram Colony,0.000000,0,22.460280,88.413810
178,Nadabhanga,0.000000,0,22.558313,88.417868
180,Naihati,0.000000,0,22.570530,88.371240
181,Naktala,0.000000,0,22.470610,88.372220
182,"Naldanga, Hooghly",0.000000,0,22.906850,88.369380
183,Nalpur,0.000000,0,22.570530,88.371240


In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = ['red','blue','darkgreen']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [130]:
rainbow

['#8000ff', '#80ffb4', '#ff0000']

In [126]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### 9. Examine Clusters

<b>Cluster 0</b>

In [127]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
141,Kamalgazi,0.0,0,22.447295,88.390689
174,Mankundu,0.0,0,22.471185,88.395153
175,"Model Town, Kolkata",0.0,0,22.463000,88.395820
176,"Mukundapur, Kolkata",0.0,0,22.589113,88.355020
177,Nabagram Colony,0.0,0,22.460280,88.413810
178,Nadabhanga,0.0,0,22.558313,88.417868
180,Naihati,0.0,0,22.570530,88.371240
181,Naktala,0.0,0,22.470610,88.372220
182,"Naldanga, Hooghly",0.0,0,22.906850,88.369380
183,Nalpur,0.0,0,22.570530,88.371240


<b>Cluster 1</b>

In [128]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
235,Sahaganj,0.166667,1,22.490805,88.393503
216,Pujali,0.142857,1,22.491383,88.388110
179,Nagerbazar,0.200000,1,22.622761,88.414315
6,Ankurhati,0.200000,1,22.487830,88.384644
262,Survey Park,0.200000,1,22.488350,88.391350
13,Baguiati,0.250000,1,22.616740,88.426910
230,Ranikuthi,0.250000,1,22.614730,88.423450
2,Ajoy Nagar,0.142857,1,22.489660,88.396400
171,Makardaha,0.166667,1,22.504850,88.406377
152,Kestopur,0.166667,1,22.603410,88.423330


<b>Cluster 2</b>

In [129]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
131,Jodhpur Park,0.083333,2,22.505860,88.363650
40,"Belur, West Bengal",0.066667,2,22.545300,88.361030
146,Kankurgachi,0.037037,2,22.578340,88.393860
149,Katju Nagar,0.040000,2,22.498400,88.369970
107,Golf Green,0.045455,2,22.494170,88.363310
164,Lake Gardens,0.041667,2,22.504930,88.356250
101,Gariahat Road,0.033333,2,22.515411,88.367789
82,Dhakuria,0.111111,2,22.510840,88.372580
76,Chowringhee,0.033333,2,22.544980,88.354340
199,North Kolkata,0.080000,2,22.503290,88.366400


## Observation

Most of the shopping malls are concentrated in the central east, southern and few on the northern part of Kolkata, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central eastern and southern part of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.